In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import time
import datetime
import re
import urllib
from bs4 import BeautifulSoup
from random import randint
import pyLDAvis.gensim
from pprint import pprint

In [3]:
initial_date_str = '1 Jan 2018'
initial_date = datetime.datetime.strptime(initial_date_str, '%d %b %Y')
final_date_str = '13 Jun 2018'

final_date = datetime.datetime.strptime(final_date_str, '%d %b %Y')
current_date = initial_date

In [4]:
tweets_list = []
while current_date < final_date :        
    aux_list = open('Tweets/' + current_date.strftime("%Y-%m-%d") + '_trump.txt',encoding='UTF-8').read().split('\n')
    aux_list = [current_date.strftime("%Y-%m-%d") + ', ' + row for row in aux_list]
    tweets_list.extend(aux_list)
    current_date = current_date + datetime.timedelta(days=1)

In [5]:
tweets_list[6500]

'2018-04-15, JTStevison, https://twitter.com/JTStevison/status/985656472687259649, 1, 11, 4, Perfect. <img class="Emoji Emoji--forText" src="https://abs.twimg.com/emoji/v2/72x72/1f602.png" draggable="false" alt="😂" title="Face with tears of joy" aria-label="Emoji: Face with tears of joy"><img class="Emoji Emoji--forText" src="https://abs.twimg.com/emoji/v2/72x72/1f602.png" draggable="false" alt="😂" title="Face with tears of joy" aria-label="Emoji: Face with tears of joy"><a href="/hashtag/DonaldTrump?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b><strong>DonaldTrump</strong></b></a> <a href="/hashtag/Syria?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>Syria</b></a> <a href="/hashtag/SyrianStrikes?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>SyrianStrikes</b></a> <a href="/hashtag/Trump?src=hash" data-qu

In [6]:
def add_space_between_a_tags(html):
    pos_1 = html.find('<a')
    while pos_1 != -1:
        html = html[:pos_1] + ' ' + html[pos_1:]
        pos_1 = html.find('<a',pos_1+2)
        
    pos_1 = html.find('</a>')
    while pos_1 != -1:
        html = html[:pos_1+4] + ' ' + html[pos_1+4:]
        #if len(html) == html.find('</a>',pos_1 + 4)
        pos_1 = html.find('</a>',pos_1+4)
        
    return html

In [7]:
print(add_space_between_a_tags(tweets_list[9870]))

2018-06-09, reggaerapstarCP, https://twitter.com/reggaerapstarCP/status/1005393266530021379, 0, 1, 3, Remember this?This This  <a href="/hashtag/Original?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>Original</b></a>   <a href="/hashtag/hand?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>hand</b></a>   <a href="/hashtag/painting?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>painting</b></a>   <a href="/hashtag/handdesigned?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>handdesigned</b></a>   <a href="/hashtag/usa?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>usa</b></a>   <a href="/hashtag/presidenttrump?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link j

In [8]:
print(tweets_list[9870])

2018-06-09, reggaerapstarCP, https://twitter.com/reggaerapstarCP/status/1005393266530021379, 0, 1, 3, Remember this?This This <a href="/hashtag/Original?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>Original</b></a> <a href="/hashtag/hand?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>hand</b></a> <a href="/hashtag/painting?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>painting</b></a> <a href="/hashtag/handdesigned?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>handdesigned</b></a> <a href="/hashtag/usa?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>usa</b></a> <a href="/hashtag/presidenttrump?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir=

In [9]:
def get_hashtags(tweet):
    hashtags = []
    pos = tweet.find('#')
    while pos != -1:
        pos_aux = tweet.find(" ",pos)
        if pos_aux == -1:
            break
        hashtags.append(tweet[pos:pos_aux])
        tweet = tweet[:pos] + ' ' + tweet[pos_aux:]
        pos = tweet.find('#',pos + 1)
    return hashtags

In [10]:
print(get_hashtags("Body Language Analysis No.4303: Donald Trump, G7 Summit, Women's Empowerment, and Rage -  #Nonverbal &  #EmotionalIntelligence •   •  #DonaldTrump #AngelaMerkel #Anger #BehaviorAnalysis #BodyLanguage #BodyLanguageExpert #Analysis #EQ #G7 #G7Summit"))

['#Nonverbal', '#EmotionalIntelligence', '#DonaldTrump', '#AngelaMerkel', '#Anger', '#BehaviorAnalysis', '#BodyLanguage', '#BodyLanguageExpert', '#Analysis', '#EQ', '#G7']


In [11]:
def get_mentions(tweet):
    mentions = []
    pos = tweet.find('@')
    while pos != -1:
        pos_aux = tweet.find(" ",pos)
        if pos_aux == -1:
            break
        mentions.append(tweet[pos:pos_aux])
        tweet = tweet[:pos] + ' ' + tweet[pos_aux:]
        pos = tweet.find('@',pos + 1)
    return mentions

In [12]:
print(get_mentions("May God Bless you and Bless God for  #DonaldTrump @lindathomas102 @louisa_ip @maga_417 @MarcGriff89 @MarjorieLoy1@Maximus_Paulus @MC_Mad_Man @mgw81180 @MrSteveRangel @MyMountainTop @Namashary @netteKnows @NotUnderdog @NYKchi88 @nykon187@ofarther"))

['@lindathomas102', '@louisa_ip', '@maga_417', '@MarcGriff89', '@MarjorieLoy1@Maximus_Paulus', '@MC_Mad_Man', '@mgw81180', '@MrSteveRangel', '@MyMountainTop', '@Namashary', '@netteKnows', '@NotUnderdog', '@NYKchi88']


In [13]:
dates = []
user = []
status = []
comments = []
likes = []
retweets = []
tweet = []
tweet_no_html = []
hashtags = []
mentions = []
for tweet_status in tweets_list:
    pos = tweet_status.find(',')
    dates.append(tweet_status[:pos])
    
    tweet_status = tweet_status[pos + 1:].strip()
    pos = tweet_status.find(',')
    user.append(tweet_status[:pos])
    
    tweet_status = tweet_status[pos + 1:].strip()
    pos = tweet_status.find(',')
    status.append(tweet_status[:pos])
    
    tweet_status = tweet_status[pos + 1:].strip()
    pos = tweet_status.find(',')
    comments.append(tweet_status[:pos])
    
    tweet_status = tweet_status[pos + 1:].strip()
    pos = tweet_status.find(',')
    likes.append(tweet_status[:pos])
    
    tweet_status = tweet_status[pos + 1:].strip()
    pos = tweet_status.find(',')
    retweets.append(tweet_status[:pos])
        
    tweet_status = tweet_status[pos + 1:].strip()    
    soup = BeautifulSoup(add_space_between_a_tags(tweet_status),'lxml')
    
    tweet_status_no_html = soup.text
    
    tweet.append(tweet_status_no_html)
    
    pos_2 = tweet_status_no_html.find('pic.twitter.com/')
    tweet_status_no_html = tweet_status_no_html[:pos_2].strip()
    pos_3 = tweet_status_no_html.find('http')
    while pos_3 != -1:        
        tweet_status_no_html = tweet_status_no_html[:pos_3 - 1] + ' ' + tweet_status_no_html[pos_3:]
        pos_3 = tweet_status_no_html.find('http',pos_3 + 1)
    tweet_status_no_html = re.sub(r"http\S+", "", tweet_status_no_html)
    tweet_status_no_html = tweet_status_no_html.strip().replace("\xa0","")
    tweet_no_html.append(tweet_status_no_html)
    
    hashtags.append(get_hashtags(tweet_status_no_html))
    mentions.append(get_mentions(tweet_status_no_html))

In [14]:
dates[1]

'2018-01-01'

# Removing short tweets

In [15]:
count = 0
index = 0
for t in tweet_no_html:
    f = t
    if len(f.lower().replace("donaldtrump","").replace(" ","").replace(".","")) < 150:
        del(dates[index])
        del(user[index])
        del(status[index])
        del(comments[index])
        del(likes[index])
        del(retweets[index])
        del(tweet[index])
        del(tweet_no_html[index])
        del(hashtags[index])
        del(mentions[index])
        count = count + 1
    index = index + 1
print(count)
print(len(dates))    

3851
6393


In [16]:
#tweet_no_html

In [17]:
x = randint(0, len(dates))
print(x)
print(dates[x],user[x],status[x],comments[x],likes[x],retweets[x],tweet[x],tweet_no_html[x],hashtags[x],mentions[x],end="\n")

842
2018-01-19 littleredblog https://twitter.com/littleredblog/status/954142541582135296 0 2 6 #HumanRightsWatch  gives a scathing assessment of  #DonaldTrump https://www.salon.com/2018/01/18/human-rights-watch-gives-a-scathing-assessment-of-donald-trump/#.WmE2CBqg6ww.twitter …  #HumanRightsWatch  gives a scathing assessment of  #DonaldTrump … ['#HumanRightsWatch', '#DonaldTrump'] []


In [18]:
index = 0
for i in tweet_no_html:
    print(index,':',i)
    index = index + 1

0 : #America  demands  #TrumpShowUsYourPenis #DonaldTrump #MAGA
1 : #DonaldTrump  stop blaming PAK... and PAK Should  #StopAmericanSupplies
2 : #DonaldTrump #DonaldTrumpsPak  Short term memory loss syndrome OR Bipolar disorder?
3 : Dear  #DonaldTrump  you know fairly after March 2018.......you will never be able to threatened State of Pakistan ever again.......So now  #NoMoreTrump
4 : Only problem is .... #DonaldTrump  can't be disappeare
5 : Becoz we are busy in lifting "Mukka" from Mukka chawk..all 33 billion dollars are used to vanish MQM frm Karachiwe need more coz MQM is still there.dont say No mOre! #DonaldTrump
6 : “The markets are happy”: Why Wall Street didn’t panic about democracy under Trump - Vox (Because they knew it’s their chance to get rich while the rest of Americans suffer at the hands of the  #GOP  and  #DonaldTrump . Insider trading anyone?) …
7 : ASSH*LE  #LordAdonis : “ #Brexit  is a  #populist  and  #nationalist  spasm worthy of  #DonaldTrump .” No,  @Andrew_Adon

954 : I am honestly starting to think that  #DonaldTrump  absent mindedness may be a result of a long standing case of the clap compounded with some sort of herpes induced crotch rot. Dementia is a symptom of syphilis. #stormytrump #StormyDaniels2020
955 : The JUST Journal  is out!  …  Stories via  @mmyer1018 @Adampdx @rcarr57 #usrc #donaldtrump
956 : Sadiq Khan compares  #DonaldTrump  to Isis: 'The president is amplifying that message of hate' #TrumpShutdown #ShitholePresident #ImpeachTrump …
957 : #ItMakesMeAnxiousWhen Donald Trump is doing anything OTHER THAN SLEEPING.When he is awake we are in trouble. Too bad we only get 4-5 hours of peace while he’s sleeping.There is a serious problem with this. #Trump  #DonaldTrump #TrumpShutDown #TrumpIsARacist #Impeach45
958 : Nice to meet you, JoeI am just an ordinary American that like so many others was fed up with the Government overreach of the Obama Administration! To me  #DonaldTrump  represents the dawn of a new day in America
959 : Ou

1954 : Ronan Farrow on another Trump affair: "My story suggests a concerted effort and a well-oiled machine designed to conceal an alleged Trump affair.  … #donaldtrump
1955 : #DonaldTrump 's ass is grass &  #RobertMueller  is the lawnmower! It has been established that Russia indeed meddled in Election 2016.  It is slowly coming together.   #LockThemAllUp #TraitorTrump #RussiaCollusion
1956 : THANK YOU  #RobertMueller  ....  AMERICA DESERVES SO MUCH BETTER THAN  #DonaldTrump  !!!!! WHATEVER THE OUTCOME....YOU AND YOUR TEAM WILL ALL GO DOWN IN HISTORY AS AMERICAN" HERO'S" !G-D BLESS YOU  #TrumpColluded #FridayFeeling
1957 : The  #Russians  even paid  #Nigerians  to demonize  @HillaryClinton  and post outright lies about her online. They gave money to the  #NRA  and  #JillStein ...Shame on  #DonaldTrump  He's betrayed American people
1958 : #DonaldTrump  still doesn't get itThe  #FloridaSchoolShooting  happens & trump ignores the need for  #GunReformNow  The  #MuellerInvestigation  show

3121 : #race  and  #electoralpolitics , and the misdirected spending priorities of the  #DemocraticParty  Steve Phillips  #author  BROWN IS THE NEW WHITE TALKish with Halli Casser-Jayne  #identitypolitics #DonaldTrump #Latino #AfricanAmerican
3122 : Republican senators warned  #US  president  #DonaldTrump  on Sunday not to fire Special Counsel  #RobertMueller …
3123 : Declassified  #CIA  Report Exposes 25 Years of  #US  Plans to Destabilize  #Syria … #Syria #Ghouta #CNN #BBC #FoxNews #ABC #NBC #CBS #DonaldTrump #NikkiHaley #MikePence #Israel #Netanyahu #EmmanuelMacron #Putin #Assad #BoycottIsrael #BDS
3124 : hell NO,it belongs to all of US...POS lefts weridos #q #qanon #Qanon8chan #Qanon4chan #clintonemails #FolllowTheWhiteRabbit #8chan #4chan #MegaAnon #mega #DonaldTrump #FreeTheInternet #POTUS #InternetBillOfRights …
3125 : Eisenhower is rolling over in his grave  #TrumpRussia #Mueller #ElectionFraud #Resist #TrumpRussiaCollusion #TrumpIsNotAboveTheLaw #DonaldTrump #Drumpf #DemForce 

4121 : ...Some people have no patience! #Putin #DonaldTrump #War #SyriaCrisis Cartoon:  @theheraldsun #Trump #TrumpRussia #Siria #SyriaStrikes #Syria #SyriaAttack #KimJongUn @cartoonmovement @procartoonists
4122 : That is your cult fantasy  #DonaldTrump . This is the reality.
4123 : @twitter  WHEN r u going to suspend  #donaldtrump  for 24-48 hours 4 attacking people on ur platform ?!  #TwitterIsComplicit
4124 : Some seem surprised that  #DonaldTrump  ‘embraced’ the ‘ #MissionAccomplished ’ statement after  #America  called him on its foolishness and lack of foresight and intelligence. Is anyone surprised?  #Trump  NEVER admits a mistake or lack of intelligence! 5-year olds never do! Disgrace
4125 : "Tax Reform" turns out to be a gift that focuses more on the very wealthy, enabled by our legislators to pay back their donors. Most of us aren't included. M people won't read this or remember next time they vote.  … #donaldtrump #republican #maga #ccw247
4126 : #Missionaccomplished , Trump

5287 : So the US puts a new  #JerusalemEmbassy  up and the first countries to sign up to join are ex-communist and  #Latino .  #DonaldTrump  must just be tickled pink to keep running into these folks. How long before he wants another wall in  #Jerusalem ?  #MJGA
5288 : On its anniversary, Mueller's investigation isn't close to finished  … #Trump #news #usa #twitter #cnn #Donaldtrump
5289 : Mother, should I build the wall?Mother, should I run for president?Mother, should I trust the government?Mother, will they put me in the firing line?Ooh, ah, is it just a waste of time?  #PinkFloyd #mother #thewall #USA #DonaldTrump #Dotard
5290 : Trump saying “Surrender your nuclear weapons or we’ll arm your rebels and/or assasinate you”  #NorthKorea #KimJongUn #DonaldTrump …
5291 : The Bolton Administration Has Already Begun   #JohnBolton #Trumpforeignpolicy #DonaldTrump #Iran #NorthKorea #diplomacy #USforeignpolicy ,  #USmilitarypolicy …
5292 : “One of they key problems today is that politics is s

In [19]:
import nltk
import re
import time
from collections import defaultdict
from configparser import ConfigParser
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from string import digits
import mpld3
import numpy as np
import pandas as pd
nltk.download('punkt')
stemmer = SnowballStemmer('english')
stopword = nltk.corpus.stopwords.words('english')
stopword = stopword + ['trump','donaldtrump','donald','realdonaldtrump']
stopword = stopword + ['usa','president','potus','us','america']

[nltk_data] Downloading package punkt to /home/miguel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [21]:
totalvocab_stemmed = []
totalvocab_tokenized = []
tweets_day = []
for t in tweet_no_html:
    i = "".join(t).replace("."," ").replace("\'","")
    tweets_day.append(i)
    allwords_stemmed = tokenize_and_stem(i) 
    totalvocab_stemmed.extend(allwords_stemmed)
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [22]:
totalvocab_tokenized

['america',
 'demands',
 'trumpshowusyourpenis',
 'donaldtrump',
 'maga',
 'donaldtrump',
 'stop',
 'blaming',
 'pak',
 'and',
 'pak',
 'should',
 'stopamericansupplies',
 'donaldtrump',
 'donaldtrumpspak',
 'short',
 'term',
 'memory',
 'loss',
 'syndrome',
 'or',
 'bipolar',
 'disorder',
 'dear',
 'donaldtrump',
 'you',
 'know',
 'fairly',
 'after',
 'march',
 'you',
 'will',
 'never',
 'be',
 'able',
 'to',
 'threatened',
 'state',
 'of',
 'pakistan',
 'ever',
 'again',
 'so',
 'now',
 'nomoretrump',
 'only',
 'problem',
 'is',
 'donaldtrump',
 'cant',
 'be',
 'disappeare',
 'becoz',
 'we',
 'are',
 'busy',
 'in',
 'lifting',
 'mukka',
 'from',
 'mukka',
 'chawk',
 'all',
 'billion',
 'dollars',
 'are',
 'used',
 'to',
 'vanish',
 'mqm',
 'frm',
 'karachiwe',
 'need',
 'more',
 'coz',
 'mqm',
 'is',
 'still',
 'there',
 'dont',
 'say',
 'no',
 'more',
 'donaldtrump',
 'the',
 'markets',
 'are',
 'happy',
 'why',
 'wall',
 'street',
 'didn',
 't',
 'panic',
 'about',
 'democracy',
 '

In [23]:
tweets_day[:20]

['#America  demands  #TrumpShowUsYourPenis #DonaldTrump #MAGA',
 '#DonaldTrump  stop blaming PAK    and PAK Should  #StopAmericanSupplies',
 '#DonaldTrump #DonaldTrumpsPak  Short term memory loss syndrome OR Bipolar disorder?',
 'Dear  #DonaldTrump  you know fairly after March 2018       you will never be able to threatened State of Pakistan ever again       So now  #NoMoreTrump',
 'Only problem is      #DonaldTrump  cant be disappeare',
 'Becoz we are busy in lifting "Mukka" from Mukka chawk  all 33 billion dollars are used to vanish MQM frm Karachiwe need more coz MQM is still there dont say No mOre! #DonaldTrump',
 '“The markets are happy”: Why Wall Street didn’t panic about democracy under Trump - Vox (Because they knew it’s their chance to get rich while the rest of Americans suffer at the hands of the  #GOP  and  #DonaldTrump   Insider trading anyone?) …',
 'ASSH*LE  #LordAdonis : “ #Brexit  is a  #populist  and  #nationalist  spasm worthy of  #DonaldTrump  ” No,  @Andrew_Adonis 

In [24]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)

In [25]:
vocab_frame.head

<bound method NDFrame.head of                                     words
america                           america
demand                            demands
trumpshowusyourpeni  trumpshowusyourpenis
donaldtrump                   donaldtrump
maga                                 maga
donaldtrump                   donaldtrump
stop                                 stop
blame                             blaming
pak                                   pak
and                                   and
pak                                   pak
should                             should
stopamericansuppli   stopamericansupplies
donaldtrump                   donaldtrump
donaldtrumpspak           donaldtrumpspak
short                               short
term                                 term
memori                             memory
loss                                 loss
syndrom                          syndrome
or                                     or
bipolar                           bipolar
diso

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8,
                                 min_df=0.015, stop_words=stopword,
                                 use_idf=True, tokenizer=tokenize_and_stem)

%time tfidf_matrix = tfidf_vectorizer.fit_transform(tweets_day) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

CPU times: user 4.39 s, sys: 3.88 ms, total: 4.39 s
Wall time: 4.39 s
(6393, 139)


In [27]:
terms = tfidf_vectorizer.get_feature_names()

In [28]:
from sklearn.cluster import KMeans

num_clusters = 25

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 2.65 s, sys: 0 ns, total: 2.65 s
Wall time: 2.65 s


In [29]:
len(tweets_day)

6393

In [30]:
tweets_info = { 'status': tweets_day, 'hashtags': hashtags, 'date': dates, 'mentions': mentions}

frame = pd.DataFrame(tweets_info, index = [clusters],columns=['status','hashtags','date','mentions','cluster'])

In [31]:
#print(frame['cluster'].value_counts())

In [32]:
def get_week_from_date(date_string):
    datetime_object = datetime.datetime.strptime(date_string,'%Y-%m-%d')
    return datetime_object.isocalendar()[1]

In [37]:
from __future__ import print_function
from collections import Counter

print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        print(' %s' % vocab_frame.loc[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    print() #add whitespace
    print() #add whitespace
    
    print("Cluster %d hashtags:" % i, end='')
    h_list_cluster = []
    for h_list in frame.loc[i]['hashtags'].values.tolist():
        h_list_cluster  = h_list_cluster + h_list
    #remove hashtag search criteria
    h_list_cluster = [ t for t in h_list_cluster if t.lower() != "#donaldtrump" and t.lower() != "#trump" and t.lower() != "#potus"]
    pprint(Counter(h_list_cluster).most_common(10))
    
    dates_list = frame.loc[i]['date'].values.tolist()
    dates_list = ['semana ' + str(get_week_from_date(t)) for t in dates_list]
    pprint(Counter(dates_list).most_common(6))
    print() #add whitespace
    print() #add whitespace
    
print()
print()

Top terms per cluster:

Cluster 0 words: b'leaders', b'president', b'meeting', b'kimjongun', b'world', b'said',

Cluster 0 hashtags:[('#KimJongUn', 17),
 ('#newsday', 8),
 ('#TheresaMay', 6),
 ('#NorthKorea', 6),
 ('#DPRK', 5),
 ('#VladimirPutin', 5),
 ('#Taliban', 4),
 ('#MikePence', 4),
 ('#JeremyCorbyn', 4),
 ('#BorisJohnson', 4)]
[('semana 1', 13),
 ('semana 18', 11),
 ('semana 10', 9),
 ('semana 24', 9),
 ('semana 4', 8),
 ('semana 21', 7)]


Cluster 1 words: b'people', b'americans', b'like', b'year', b'president', b'say',

Cluster 1 hashtags:[('#MAGA', 13),
 ('#HillaryClinton', 5),
 ('#WhiteHouse', 5),
 ('#Republicans', 4),
 ('#ImpeachTrump', 4),
 ('#USA', 4),
 ('#Iran', 3),
 ('#FoxNews', 3),
 ('#CIA', 3),
 ('#Democrats', 3)]
[('semana 23', 18),
 ('semana 6', 17),
 ('semana 1', 16),
 ('semana 3', 15),
 ('semana 7', 14),
 ('semana 15', 14)]


Cluster 2 words: b'making', b'maga', b'great', b'president', b'look', b'one',

Cluster 2 hashtags:[('#MAGA', 30),
 ('#humor', 18),
 ('#snl',

 ('#National', 4),
 ('#JohnBolton', 4),
 ('#BuildTheWall', 3),
 ('#AmericaFirst', 3),
 ('#UK', 3),
 ('#passiveincome', 3)]
[('semana 5', 13),
 ('semana 2', 12),
 ('semana 15', 11),
 ('semana 21', 7),
 ('semana 4', 6),
 ('semana 12', 6)]


Cluster 23 words: b'via', b'calls', b'political', b'new', b'president', b'got',

Cluster 23 hashtags:[('#NorthKorea', 5),
 ('#USA', 5),
 ('#law', 4),
 ('#marketing', 4),
 ('#HumanRights', 4),
 ('#SteveBannon', 3),
 ('#GOP', 3),
 ('#BreakingNews', 3),
 ('#RobertMueller', 3),
 ('#Dubai', 3)]
[('semana 16', 9),
 ('semana 18', 9),
 ('semana 7', 8),
 ('semana 19', 7),
 ('semana 5', 6),
 ('semana 10', 6)]


Cluster 24 words: b'u', b'president', b'state', b'unite', b'say', b'look',

Cluster 24 hashtags:[('#USA', 5),
 ('#MAGA', 5),
 ('#NorthKorea', 5),
 ('#KimJongUn', 4),
 ('#StormyDaniels', 3),
 ('#GOP', 3),
 ('#Iran', 3),
 ('#Denuclearization', 3),
 ('#realDonaldTrump', 2),
 ('#traitor', 2)]
[('semana 3', 8),
 ('semana 13', 7),
 ('semana 14', 7),
 ('semana 